<a href="https://colab.research.google.com/github/noriakihanya/musicgen/blob/main/AudioCraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Adapted from https://github.com/camenduru/MusicGen-colab
%cd /content
!git clone https://github.com/facebookresearch/audiocraft
%cd /content/audiocraft
!pip install -r requirements.txt
# Click on the gradio link that appear.
!python app.py --share
# See also https://colab.research.google.com/drive/1fxGqfg96RBUvGxZ1XXN07s3DthrKUl4-?usp=sharing
# for a Colab demo using the underlying API instead.

In [1]:
!pip install -U audiocraft
!pip install mutagen
!git clone https://github.com/facebookresearch/audiocraft
!pip install pydub

import gdown
gdown.download('https://drive.google.com/uc?id=1wVauTFZFFCqbiFzRAK2Th_Kn0p5HzOiG', 'MuseScore_General.sf2', quiet=False)
import shutil
!git clone https://github.com/noriakihanya/musicgen.git
shutil.copy("/content/musicgen/musicgen.py","/usr/local/lib/python3.10/dist-packages/audiocraft/models")

import os
os.makedirs("/content/00_tmp/", exist_ok=True)
os.makedirs("/content/01_input", exist_ok=True)
os.makedirs("/content/02_output_midi", exist_ok=True)
os.makedirs("/content/03_output_mp3", exist_ok=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.1/259.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 86.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.0 MB/s eta 0:00:00
Cloning into 'audiocraft'...
remote: Enumerating objects: 461, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 461 (delta 188), reused 151 (delta 141), pack-reused 228
Receiving objects: 100% (461/461), 658.06 KiB | 5.93 MiB/s, done.
Resolving deltas: 100% (275/275), done.


Downloading...
From: https://drive.google.com/uc?id=1wVauTFZFFCqbiFzRAK2Th_Kn0p5HzOiG
To: /content/MuseScore_General.sf2
100%|██████████| 216M/216M [00:01<00:00, 113MB/s] 


Cloning into 'musicgen'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 30 (delta 12), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (30/30), 4.28 MiB | 8.84 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [6]:
#@title MusicGen API
import os
import shutil
import glob
import math
import torch
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from audiocraft.data.audio_utils import convert_audio
from mutagen.mp3 import MP3
from pydub import AudioSegment

#for file in glob.glob('/content/03_output_mp3/*.mp3'):
#    os.remove(file)

#for file in glob.glob('/content/01_input/*.mp3'):
#  shutil.copy(file,"/content/03_output_mp3/")

for file in glob.glob('/content/01_input/*'):
    print(file)
    if ".wav"  in file:
      # 読み込み
      sound = AudioSegment.from_wav(file)
      # 変換
      file = "/content/01_input/musicgen.mp3"
      sound.export(file, format="mp3")

audio_time = MP3(file)
model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=audio_time.info.length)  # generate 8 seconds.
#model.set_generation_params(10)
#descriptions = ['An 80s driving pop song with heavy drums and synth pads in the background']
description = "Accompaniment to fit an 80s driving pop song" #@param {type:"string"}
descriptions = [description]

# generates using the melody from the given audio and the provided descriptions.
melody, sr = torchaudio.load(file)



if melody.dim() == 1:
  melody = melody[None]
melody = melody[..., :int(sr * audio_time.info.length)]

target_sr = 32000
target_ac = 1
melody = convert_audio(melody, sr, target_sr, target_ac)

wav = model.generate_with_chroma(descriptions, melody[None].expand(len(descriptions), -1, -1), target_sr, progress=True)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write('/content/03_output_mp3/acp_musicgen', one_wav.detach().cpu().float(), model.sample_rate,format="mp3",strategy="loudness")

audio = AudioSegment.from_file('/content/03_output_mp3/acp_musicgen.mp3')
# 音量(%)
volume_percent = 350

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export('/content/03_output_mp3/acp_musicgen.mp3', format='mp3')

import IPython.display
IPython.display.Audio('/content/03_output_mp3/acp_musicgen.mp3')

/content/01_input/singingai_world.wav


Downloading: "https://dl.fbaipublicfiles.com/audiocraft/musicgen/v0/b0dbef54-37d256b525.th" to /root/.cache/torch/hub/checkpoints/b0dbef54-37d256b525.th
100%|██████████| 225M/225M [00:01<00:00, 122MB/s]
Downloading: "https://dl.fbaipublicfiles.com/audiocraft/musicgen/v0/f79af192-61305ffc49.th" to /root/.cache/torch/hub/checkpoints/f79af192-61305ffc49.th
100%|██████████| 2.58G/2.58G [00:31<00:00, 87.7MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 133MB/s]


CLIPPING /content/03_output_mp3/acp_musicgen happening with proba (a bit of clipping is okay): 0.0014073014026507735 maximum scale:  2.3120405673980713


In [ ]:
import math
audio = AudioSegment.from_file('/content/03_output_mp3/acp_musicgen.mp3')
# 音量(%)
volume_percent = 350

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export('/content/03_output_mp3/acp_musicgen.mp3', format='mp3')

import IPython.display
IPython.display.Audio('/content/03_output_mp3/acp_musicgen.mp3')

In [ ]:
#@title インストール
# Best to make sure you have torch installed first, in particular before installing xformers.
# Don't run this if you already have PyTorch installed.
!pip install 'torch>=2.0'
# Then proceed to one of the following
!pip install -U audiocraft  # stable release
!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft  # bleeding edge
!pip install -e .  # or if you cloned the repo locally

!git clone https://github.com/facebookresearch/audiocraft

In [ ]:
#@title MusicGen API
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=30)  # generate 8 seconds.

#wav = model.generate_unconditional(4)    # generates 4 unconditional audio samples

#descriptions = ['happy rock', 'energetic EDM', 'sad jazz']
#wav = model.generate(descriptions)  # generates 3 samples.

melody, sr = torchaudio.load('/content/audiocraft/assets/bach.mp3')
#descriptions = ['happy rock', 'energetic EDM', 'sad jazz']
descriptions = ['acid jazz']
# generates using the melody from the given audio and the provided descriptions.
wav = model.generate_with_chroma(descriptions, melody[None].expand(len(descriptions), -1, -1), sr)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

CLIPPING 0 happening with proba (a bit of clipping is okay): 0.0007479166379198432 maximum scale:  1.204222559928894


In [ ]:
#@title UI付き
# Adapted from https://github.com/camenduru/MusicGen-colab
%cd /content
!git clone https://github.com/facebookresearch/audiocraft
%cd /content/audiocraft
!pip install -r requirements.txt
# Click on the gradio link that appear.
!python app.py --share
# See also https://colab.research.google.com/drive/1fxGqfg96RBUvGxZ1XXN07s3DthrKUl4-?usp=sharing
# for a Colab demo using the underlying API instead.

In [ ]:
from audiocraft.models import MusicGen

# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('small')

In [ ]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=5
)

In [ ]:
#@title Unconditional Generation
from audiocraft.utils.notebook import display_audio

output = model.generate_unconditional(num_samples=2, progress=True)
display_audio(output, sample_rate=32000)

In [ ]:
#@title Music Continuation
import math
import torchaudio
import torch
from audiocraft.utils.notebook import display_audio

def get_bip_bip(bip_duration=0.125, frequency=440,
                duration=0.5, sample_rate=32000, device="cuda"):
    """Generates a series of bip bip at the given frequency."""
    t = torch.arange(
        int(duration * sample_rate), device="cuda", dtype=torch.float) / sample_rate
    wav = torch.cos(2 * math.pi * 440 * t)[None]
    tp = (t % (2 * bip_duration)) / (2 * bip_duration)
    envelope = (tp >= 0.5).float()
    return wav * envelope

In [ ]:
# Here we use a synthetic signal to prompt both the tonality and the BPM
# of the generated audio.
res = model.generate_continuation(
    get_bip_bip(0.125).expand(2, -1, -1),
    32000, ['Jazz jazz and only jazz',
            'Heartful EDM with beautiful synths and chords'],
    progress=True)
display_audio(res, 32000)

In [ ]:
# You can also use any audio from a file. Make sure to trim the file if it is too long!
prompt_waveform, prompt_sr = torchaudio.load("./assets/bach.mp3")
prompt_duration = 2
prompt_waveform = prompt_waveform[..., :int(prompt_duration * prompt_sr)]
output = model.generate_continuation(prompt_waveform, prompt_sample_rate=prompt_sr, progress=True)
display_audio(output, sample_rate=32000)

In [ ]:
#@title Text-conditional Generation
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    progress=True
)
display_audio(output, sample_rate=32000)

In [ ]:
#@title Melody-conditional Generation
import torchaudio
from audiocraft.utils.notebook import display_audio

model = MusicGen.get_pretrained('melody')
model.set_generation_params(duration=8)

melody_waveform, sr = torchaudio.load("assets/bach.mp3")
melody_waveform = melody_waveform.unsqueeze(0).repeat(2, 1, 1)
output = model.generate_with_chroma(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True
)
display_audio(output, sample_rate=32000)